# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [5]:
!pip install pandarallel


In [6]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

/kaggle/input/30-min-windowed-data/window_df (1).pkl
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/later-data/Full_merged.pkl
/kaggle/input/later-data/full_clustered .pkl
/kaggle/input/later-data/cluster_model.joblib
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [5]:
df=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [ ]:
events['step'].isnull().sum()

In [ ]:
event_df=events.dropna()

In [ ]:
event_df

In [6]:
df['window']=np.nan

In [7]:
ind=df.index[df['event'].isna()==False]
print(ind)

Index([     4992,     10932,     20244,     27492,     39996,     44400,
           57240,     62856,     91296,     97860,
       ...
       127859076, 127865244, 127876812, 127883064, 127892916, 127901112,
       127910520, 127914564, 127928580, 127935564],
      dtype='int64', length=9585)


In [8]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-179
    b=i+180
    df['window'].loc[a:b]=int(c)
    c=c+1
    



100%|██████████| 9585/9585 [00:03<00:00, 3014.50it/s]


In [9]:
df_win=df[df['window'].isna()==False]

In [2]:
df_win['window'].isnull().sum()

NameError: name 'df_win' is not defined

In [ ]:
df.to_pickle('ts_window_df.pkl')
#df=pd.read_pickle('/kaggle/input/30-min-windowed-data/window_df (1).pkl')

In [8]:
df

,series_id,step,timestamp,anglez,enmo,event,sleep,cluster,window
4813,038441c925bb,4813,2018-08-14T22:11:05-0400,22.208799,0.0035,NaN,0.0,0,0.0
4814,038441c925bb,4814,2018-08-14T22:11:10-0400,22.190399,0.0030,NaN,0.0,0,0.0
4815,038441c925bb,4815,2018-08-14T22:11:15-0400,24.370899,0.0065,NaN,0.0,0,0.0
4816,038441c925bb,4816,2018-08-14T22:11:20-0400,36.879700,0.0001,NaN,0.0,0,0.0
4817,038441c925bb,4817,2018-08-14T22:11:25-0400,37.442402,0.0000,NaN,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
127935740,fe90110788d2,581780,2017-09-07T09:31:40-0400,-20.881100,0.0000,NaN,0.0,1,9584.0
127935741,fe90110788d2,581781,2017-09-07T09:31:45-0400,-22.082701,0.0000,NaN,0.0,1,9584.0
127935742,fe90110788d2,581782,2017-09-07T09:31:50-0400,-22.522200,0.0000,NaN,0.0,1,9584.0
127935743,fe90110788d2,581783,2017-09-07T09:31:55-0400,-22.522200,0.0000,NaN,0.0,1,9584.0


In [ ]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))